<a href="https://colab.research.google.com/github/RenanLM/nbagent-system/blob/main/NBAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

by Renan Moura

# **Imersão IA Alura + Google Gemini 2025**

# Chatbot para Previsão de desempenho de um jogador de basquete da NBA

In [87]:
%pip -q install google-genai google-adk

In [88]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google import genai
client = genai.Client()

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
from IPython.display import display, Markdown
import textwrap
import warnings
warnings.filterwarnings("ignore")

In [89]:
# Função para executar um agente com entrada do usuário
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

# Função para exibir o texto em formato de citação (Markdown)
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [90]:
###########################################################
# --- Agente 1: Buscador de Estatísticas de Jogadores --- #
###########################################################
def agente_coletor_dados(jogador, data_de_hoje):
    coletor = Agent(
        name="agente_coletor_dados",
        model="gemini-2.0-flash",
        tools=[google_search],
        instruction="""
        Você é um agente que coleta dados estatísticos recentes da NBA sobre o jogador fornecido.
        Utilize a busca do Google para obter informações atualizadas como pontos por jogo, assistências,
        rebotes, aproveitamento de arremessos, desempenho recente em jogos e outras métricas relevantes.

        A resposta deve ser objetiva, listando as principais estatísticas encontradas, especialmente das últimas partidas.
        Busque sempre a informação mais recente possível.

        ⚠️ Se o jogador estiver lesionado, eliminado dos playoffs ou não estiver mais participando da temporada (como em casos de eliminação ou não classificação),
        informe isso claramente e traga apenas os dados disponíveis, sem mencionar possíveis próximos jogos ou previsões futuras.
        """
    )
    entrada = f"Jogador: {jogador}\nData: {data_de_hoje}"
    dados = call_agent(coletor, entrada)
    return dados


In [91]:
#######################################################
# --- Agente 2: Buscador de Estatísticas de Times --- #
#######################################################

def agente_analista_estatistico(jogador, dados_estatisticos):
    analista = Agent(
        name="agente_analista_estatistico",
        model="gemini-2.0-flash",
        instruction="""
        Você é um analista esportivo especializado em NBA. Com base nos dados estatísticos recentes de um jogador,
        identifique tendências, como aumento ou queda de performance, padrões relevantes e fatores que impactam sua atuação.
        Se o jogador estiver lesionado, eliminado dos playoffs ou não estiver mais participando da temporada,
        destaque isso de forma clara em sua análise e baseie sua resposta apenas nos dados já ocorridos,
        sem fazer projeções futuras.

        Elabore uma análise objetiva, incluindo se a performance está em ascensão, estável ou em queda.
        """
    )
    entrada = f"Jogador: {jogador}\nDados: {dados_estatisticos}"
    analise = call_agent(analista, entrada)
    return analise

In [92]:
#######################################################
# --- Agente 3: Previsor de Desempenho de Jogador --- #
#######################################################
def agente_previsor(jogador, analise):
    previsor = Agent(
        name="agente_previsor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente previsor de desempenho na NBA. Com base na análise estatística de um jogador,
        gere uma previsão realista do seu desempenho em um próximo jogo da temporada (por exemplo: "O jogador deve marcar entre 20 e 25 pontos").
        Seja preciso, mas evite exageros. Use termos como "alta probabilidade", "tendência", "expectativa".
        Caso o jogador estiver lesionado, eliminado dos playoffs, ou não estiver mais participando da temporada,
        você pode trazer estatísticas e análises, mas não deve fazer previsões futuras de desempenho ou jogos.
        """
    )
    entrada = f"Jogador: {jogador}\nAnálise: {analise}"
    previsao = call_agent(previsor, entrada)
    return previsao

In [93]:
###################################################
# --- Agente 4: Previsor de Resultado de Jogo --- #
###################################################
def agente_validador(jogador, previsao):
    validador = Agent(
        name="agente_validador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente crítico que valida previsões estatísticas da NBA.
        Verifique se a previsão para o jogador é coerente com os dados recentes e explique em poucas frases por quê.
        Caso o jogador estiver lesionado, eliminado dos playoffs, ou não estiver mais participando da temporada,
        você pode validar mesmo assim.
        """
    )
    entrada = f"Jogador: {jogador}\nPrevisão: {previsao}"
    validacao = call_agent(validador, entrada)
    return validacao

In [95]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
jogador = input("🏀 Digite o nome do jogador da NBA para prever o desempenho: ")

if not jogador:
    print("Você esqueceu de digitar o nome do jogador!")
else:
    print(f"\n🔎 Buscando dados de {jogador}...")
    dados = agente_coletor_dados(jogador, data_de_hoje)
    display(to_markdown(dados))

    print("\n📊 Analisando estatísticas...")
    analise = agente_analista_estatistico(jogador, dados)
    display(to_markdown(analise))

    print("\n🔮 Gerando previsão...")
    previsao = agente_previsor(jogador, analise)
    display(to_markdown(previsao))

    print("\n✅ Validando previsão...")
    validacao = agente_validador(jogador, previsao)
    display(to_markdown(validacao))

🏀 Digite o nome do jogador da NBA para prever o desempenho: Nikola Jokic

🔎 Buscando dados de Nikola Jokic...


> Ok, coletarei dados estatísticos recentes sobre Nikola Jokic até 18 de maio de 2025.
> 
> 
> Com certeza! Aqui estão as estatísticas recentes de Nikola Jokic até 18 de maio de 2025:
> 
> **Estatísticas Gerais:**
> 
> *   **Temporada Regular 2024-25:** Jokic teve médias de 29.6 pontos, 12.7 rebotes e 10.2 assistências por jogo.
> *   **Aproveitamento:** 57.6% nos arremessos de quadra, 41.7% nas bolas de três e 80.0% nos lances livres.
> 
> **Desempenho nos Playoffs de 2025:**
> 
> *   Em 13 jogos nos playoffs, Jokic teve médias de 26.7 pontos, 13.0 rebotes e 8.1 assistências.
> *   O aproveitamento nos playoffs foi de 48.6% nos arremessos de quadra e 37.7% nas bolas de três.
> 
> **Jogos Recentes (Semifinais da Conferência Oeste contra o Oklahoma City Thunder):**
> 
> *   **15 de maio de 2025 (Jogo 6):** 29 pontos, 14 rebotes e 8 assistências na vitória por 119-107.
> *   **13 de maio de 2025 (Jogo 5):** 44 pontos e 15 rebotes na derrota por 112-105.
> *   **11 de maio de 2025 (Jogo 4):** 27 pontos e 13 rebotes na derrota por 92-87.
> *   **09 de maio de 2025 (Jogo 3):** 20 pontos e 16 rebotes na vitória por 113-104.
> *   **07 de maio de 2025 (Jogo 2):** 17 pontos e 8 rebotes na derrota por 149-106.
> *   **05 de maio de 2025 (Jogo 1):** 42 pontos e 22 rebotes na vitória por 121-119.
> 
> **Informações Adicionais:**
> 
> *   Nikola Jokic liderou a liga em triplos-duplos nesta temporada com 25.
> *   Ele teve um jogo histórico em 7 de março de 2025, com 31 pontos, 21 rebotes e 22 assistências contra o Phoenix Suns.
> *   No Jogo 7 contra o Oklahoma City Thunder, Nikola Jokic está saudável e pronto para jogar.
> 
> Espero que estas informações sejam úteis!
> 



📊 Analisando estatísticas...


> ## Análise de Desempenho de Nikola Jokic (até 18 de Maio de 2025)
> 
> Nikola Jokic apresenta um desempenho geral notável, tanto na temporada regular quanto nos playoffs de 2025. Sua capacidade de pontuar, rebater e distribuir a bola é excepcional, demonstrada pelas médias de quase triplo-duplo na temporada regular (29.6 pontos, 12.7 rebotes e 10.2 assistências) e pelo número expressivo de triplos-duplos (25) liderando a liga.
> 
> **Tendências e Padrões:**
> 
> *   **Temporada Regular:** Jokic manteve um alto nível de eficiência, com um aproveitamento de arremessos de quadra de 57.6% e 41.7% nas bolas de três, o que demonstra sua versatilidade e capacidade de pontuar de diferentes áreas da quadra.
> *   **Playoffs:** Apesar de uma ligeira queda na pontuação em relação à temporada regular (26.7 pontos), ele elevou sua média de rebotes (13.0) e manteve um impacto significativo no jogo. O aproveitamento de arremessos de quadra diminuiu para 48.6% e o de bolas de três para 37.7%.
> *   **Jogos Recentes (Semifinais contra OKC):** O desempenho nos jogos recentes contra o Oklahoma City Thunder demonstra uma variação significativa. Jokic alternou entre performances dominantes (44 pontos e 15 rebotes no Jogo 5, 42 pontos e 22 rebotes no Jogo 1) e jogos mais discretos (17 pontos no Jogo 2). A capacidade de ajustar seu jogo e contribuir de diferentes formas é evidente, com destaque para a eficiência nos rebotes, que se manteve constante.
> 
> **Fatores que Impactam a Atuação:**
> 
> *   **Oponente:** A variação no desempenho nos playoffs e, especificamente, nos jogos contra o Oklahoma City Thunder, sugere que a qualidade da defesa adversária tem um impacto significativo na sua pontuação e eficiência.
> *   **Volume de Jogo:** A carga de minutos e a responsabilidade de ser o principal ponto focal ofensivo da equipe podem influenciar a consistência do seu desempenho, especialmente em jogos consecutivos ou em séries de playoffs.
> 
> **Conclusão:**
> 
> O desempenho de Nikola Jokic é de um jogador de elite, com habilidades excepcionais e impacto significativo em todas as áreas do jogo. Apesar de algumas variações, seu nível geral de performance é consistente e ele se mantém como uma força dominante na NBA.
> 
> **Classificação:** Desempenho estável com potencial de ascensão dependendo da adaptação às estratégias defensivas adversárias.
> 



🔮 Gerando previsão...


> Com base na análise estatística, Nikola Jokic tem alta probabilidade de manter um desempenho de alto nível em um próximo jogo da temporada. A expectativa é que ele continue sendo o principal ponto focal ofensivo do Denver Nuggets, contribuindo com uma pontuação entre 25 e 30 pontos, além de coletar aproximadamente 12 a 14 rebotes e distribuir cerca de 9 a 11 assistências. A capacidade de Jokic de adaptar seu jogo ao oponente é um fator crucial, e a sua eficiência nos arremessos e rebotes deve permanecer constante, a menos que a defesa adversária consiga limitar seu espaço e oportunidades.



✅ Validando previsão...


> A previsão para Nikola Jokic parece coerente com seu desempenho histórico e recente. Ele consistentemente atinge estatísticas próximas ou superiores às projetadas, mantendo-se como o principal catalisador ofensivo do Denver Nuggets. Sua capacidade de pontuar, rebater e dar assistências o torna um dos jogadores mais versáteis e impactantes da NBA, justificando a previsão otimista.
